# Develop the RNN Regression Model for the Roasting Quality data. The roasting machine had 5 chambers, and there were 3 temperature sensors in each chamber.The layer height and humanity were also measured. The data was collected every one minute and saved in data_X.csv. The roasting quality was measured every one hour and saved in data_Y.csv. Show the loss of each epoch.
Hint: Some of the measurement data was missing. You have to clean the data first.
The loss can be “mse”, “mae”, or “mape” for regression problem.
Reference:
https://www.kaggle.com/code/conormanley/productqualityprediction

In [65]:
import pandas as pd
from pandas import read_csv
import numpy as np
from keras import datasets
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
import matplotlib.pyplot as plt

In [66]:
data_X = read_csv ("data_X.csv", parse_dates=['date_time'])

In [67]:
data_Y = read_csv ("data_Y.csv", parse_dates=['date_time'])

In [68]:
data_X["date_hour"] = data_X["date_time"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

In [69]:
L = list(data_X.groupby("date_hour"))
l = len(L)
List = []
for i in range(l):
    dh = L[i][1]["date_hour"].iloc[0]
    row = L[i][1].drop(["date_time", "date_hour"], axis=1).to_numpy().flatten().tolist()
    row.append(dh)
    List.append(row)
    if (i+1)%2000 == 0:
        print("Processing: %.4f%%"%(100*(i+1)/l))

df = pd.DataFrame(List)
df["date_hour"] = df[1020]
df.drop([1020], axis=1, inplace=True)

Processing: 5.7037%
Processing: 11.4074%
Processing: 17.1111%
Processing: 22.8148%
Processing: 28.5185%
Processing: 34.2222%
Processing: 39.9259%
Processing: 45.6295%
Processing: 51.3332%
Processing: 57.0369%
Processing: 62.7406%
Processing: 68.4443%
Processing: 74.1480%
Processing: 79.8517%
Processing: 85.5554%
Processing: 91.2591%
Processing: 96.9628%


In [70]:
data_Y["date_shifted"] = pd.to_datetime(data_Y["date_time"]) - datetime.timedelta(hours=1)
data_Y["date_shifted"] = pd.to_datetime(data_Y["date_shifted"])
data_Y["date_shifted"] = data_Y["date_shifted"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

In [71]:
transformed_data = pd.merge(df,data_Y[["date_shifted", "quality"]],left_on="date_hour", right_on="date_shifted", how="inner")

In [72]:
df_X = transformed_data.drop(columns=['date_hour', 'date_shifted', 'quality'])
df_Y = transformed_data.quality

In [73]:
df_X = df_X.values.reshape((df_X.shape[0], 60, 17))
df_Y = df_Y.values.reshape((df_Y.shape[0], 1))
df_X = np.asarray(df_X).astype('float32')
df_Y = np.asarray(df_Y).astype('float32')

In [74]:
X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=0)
print(X_train.shape)
print(Y_train.shape)

(23347, 60, 17)
(23347, 1)


### (a) Using RNN with 50 units.

In [80]:
model = Sequential([LSTM(units = 50, activation = 'relu', input_shape = (60,17))])
model.add(Dense(50, activation = 'relu'))
model.add(Dense(1))

model.compile(loss = 'MSE', optimizer = 'adam', metrics = ['mae'])

In [81]:
model.fit(X_train, Y_train, epochs = 40, batch_size = 64)
loss, mape = model.evaluate(X_test, Y_test)
print("loss = ", loss)
print("mape = ", mape)

Epoch 1/40
365/365 [==============================] - 6s 14ms/step - loss: 17112298.0000 - mae: 1515.8171
Epoch 2/40
365/365 [==============================] - 5s 14ms/step - loss: 164229.1875 - mae: 402.5875
Epoch 3/40
365/365 [==============================] - 5s 14ms/step - loss: 164073.2656 - mae: 402.3939
Epoch 4/40
365/365 [==============================] - 5s 14ms/step - loss: 163882.6875 - mae: 402.1568
Epoch 5/40
365/365 [==============================] - 5s 14ms/step - loss: 163665.5781 - mae: 401.8867
Epoch 6/40
365/365 [==============================] - 5s 14ms/step - loss: 163428.3125 - mae: 401.5914
Epoch 7/40
365/365 [==============================] - 5s 14ms/step - loss: 163175.7500 - mae: 401.2771
Epoch 8/40
365/365 [==============================] - 5s 14ms/step - loss: 162912.2031 - mae: 400.9484
Epoch 9/40
365/365 [==============================] - 5s 14ms/step - loss: 162640.5312 - mae: 400.6096
Epoch 10/40
365/365 [==============================] - 5s 14ms/step - 

### (b) Try LSTM with different units to achieve better performance if possible.

In [82]:
model = Sequential([LSTM(units = 100, activation = 'relu', input_shape = (60,17))])
model.add(Dense(50, activation = 'relu'))
model.add(Dense(1))

model.compile(loss = 'MSE', optimizer = 'adam', metrics = ['mae'])
model.fit(X_train, Y_train, epochs = 40, batch_size = 64)
loss, mape = model.evaluate(X_test, Y_test)
print("loss = ", loss)
print("mape = ", mape)

Epoch 1/40
365/365 [==============================] - 9s 22ms/step - loss: 438410912.0000 - mae: 6222.8643
Epoch 2/40
365/365 [==============================] - 8s 22ms/step - loss: 18889396.0000 - mae: 2162.6609
Epoch 3/40
365/365 [==============================] - 8s 22ms/step - loss: 126001.9766 - mae: 172.7282
Epoch 4/40
365/365 [==============================] - 8s 22ms/step - loss: 28016.4023 - mae: 83.4311
Epoch 5/40
365/365 [==============================] - 8s 22ms/step - loss: 16023.8750 - mae: 68.9585
Epoch 6/40
365/365 [==============================] - 8s 23ms/step - loss: 37139.7539 - mae: 106.4051
Epoch 7/40
365/365 [==============================] - 8s 22ms/step - loss: 24321.4551 - mae: 90.7869
Epoch 8/40
365/365 [==============================] - 8s 22ms/step - loss: 17240.3008 - mae: 71.9721
Epoch 9/40
365/365 [==============================] - 8s 22ms/step - loss: 15334.6328 - mae: 78.9220
Epoch 10/40
365/365 [==============================] - 8s 23ms/step - loss: 1

In [83]:
model = Sequential([LSTM(units = 150, activation = 'relu', input_shape = (60,17))])
model.add(Dense(50, activation = 'relu'))
model.add(Dense(1))

model.compile(loss = 'MSE', optimizer = 'adam', metrics = ['mae'])
model.fit(X_train, Y_train, epochs = 40, batch_size = 64)
loss, mape = model.evaluate(X_test, Y_test)
print("loss = ", loss)
print("mape = ", mape)

Epoch 1/40
365/365 [==============================] - 14s 36ms/step - loss: 172278.2031 - mae: 405.1019
Epoch 2/40
365/365 [==============================] - 13s 36ms/step - loss: 163932.6875 - mae: 402.2190
Epoch 3/40
365/365 [==============================] - 13s 36ms/step - loss: 163639.8594 - mae: 401.8550
Epoch 4/40
365/365 [==============================] - 13s 36ms/step - loss: 163347.1406 - mae: 401.4905
Epoch 5/40
365/365 [==============================] - 13s 36ms/step - loss: 163054.7031 - mae: 401.1263
Epoch 6/40
365/365 [==============================] - 13s 36ms/step - loss: 162762.6406 - mae: 400.7620
Epoch 7/40
365/365 [==============================] - 13s 36ms/step - loss: 162470.8281 - mae: 400.3976
Epoch 8/40
365/365 [==============================] - 13s 36ms/step - loss: 162179.3594 - mae: 400.0335
Epoch 9/40
365/365 [==============================] - 13s 36ms/step - loss: 161888.1250 - mae: 399.6693
Epoch 10/40
365/365 [==============================] - 13s 36ms/